In [2]:
import warnings
warnings.simplefilter('ignore')
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data_select.csv")

# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
# selected_features = df[['names', 'of', 'selected', 'features', 'here']]
disposition = df["koi_disposition"]
feature = df.drop(['koi_disposition'], axis=1)
feature.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period_err1,koi_period_err2,koi_time0bk_err1,koi_time0bk_err2,koi_impact_err1,koi_duration_err1,koi_duration_err2,koi_depth,koi_teq,koi_model_snr,koi_slogg,koi_slogg_err2,ra,dec,koi_kepmag
0,0,0,0,0,2.479000e-04,-2.479000e-04,0.003520,-0.003520,0.059,0.11600,-0.11600,874.8,443,25.8,4.467,-0.096,291.93423,48.141651,15.347
1,0,1,0,0,1.490000e-05,-1.490000e-05,0.000581,-0.000581,5.126,0.03410,-0.03410,10829.0,638,76.3,4.544,-0.176,297.00482,48.134129,15.436
2,0,1,0,0,2.630000e-07,-2.630000e-07,0.000115,-0.000115,0.115,0.00537,-0.00537,8079.2,1395,505.6,4.564,-0.168,285.53461,48.285210,15.597
3,0,0,0,0,3.760000e-06,-3.760000e-06,0.001130,-0.001130,0.235,0.04200,-0.04200,603.3,1406,40.9,4.438,-0.210,288.75488,48.226200,15.509
4,0,0,0,0,1.050000e-05,-1.050000e-05,0.001900,-0.001900,0.139,0.06730,-0.06730,686.0,1160,40.2,4.486,-0.229,296.28613,48.224670,15.714


# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
# Assign X (data) and y (target)
X = feature
y = disposition.values.reshape(-1,1)
print(X.shape, y.shape)

(6991, 19) (6991, 1)


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,stratify=y)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [8]:
# Scale your data
# Import dependencies
from sklearn.preprocessing import LabelEncoder
# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
encoded_y_train

array([2, 0, 2, ..., 2, 0, 0], dtype=int64)

In [9]:
from sklearn.preprocessing import MinMaxScaler
X_minmax = MinMaxScaler().fit(X_train)

X_train_minmax = X_minmax.transform(X_train)
X_test_minmax = X_minmax.transform(X_test)

In [10]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_minmax, encoded_y_train)
predictions = model.predict(X_test_minmax)
model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
print(f"Training Data Score: {model.score(X_train_minmax, encoded_y_train)}")
print(f"Testing Data Score: {model.score(X_test_minmax, encoded_y_test)}")

Training Data Score: 0.8458897577722678
Testing Data Score: 0.8627002288329519


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [12]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [400, 800, 1200, 2000],
              'max_iter': [25, 50, 100, 200]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [13]:
# Train the model with GridSearch
grid.fit(X_train_minmax, encoded_y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=400, max_iter=25 ..............................................
[CV] .................. C=400, max_iter=25, score=0.853, total=   0.0s
[CV] C=400, max_iter=25 ..............................................
[CV] .................. C=400, max_iter=25, score=0.862, total=   0.0s
[CV] C=400, max_iter=25 ..............................................
[CV] .................. C=400, max_iter=25, score=0.840, total=   0.0s
[CV] C=400, max_iter=25 ..............................................
[CV] .................. C=400, max_iter=25, score=0.857, total=   0.0s
[CV] C=400, max_iter=25 ..............................................
[CV] .................. C=400, max_iter=25, score=0.846, total=   0.0s
[CV] C=400, max_iter=50 ..............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] .................. C=400, max_iter=50, score=0.862, total=   0.1s
[CV] C=400, max_iter=50 ..............................................
[CV] .................. C=400, max_iter=50, score=0.876, total=   0.1s
[CV] C=400, max_iter=50 ..............................................
[CV] .................. C=400, max_iter=50, score=0.845, total=   0.1s
[CV] C=400, max_iter=50 ..............................................
[CV] .................. C=400, max_iter=50, score=0.864, total=   0.1s
[CV] C=400, max_iter=50 ..............................................
[CV] .................. C=400, max_iter=50, score=0.857, total=   0.1s
[CV] C=400, max_iter=100 .............................................
[CV] ................. C=400, max_iter=100, score=0.857, total=   0.1s
[CV] C=400, max_iter=100 .............................................
[CV] ................. C=400, max_iter=100, score=0.876, total=   0.1s
[CV] C=400, max_iter=100 .............................................
[CV] .

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    9.4s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [400, 800, 1200, 2000],
                         'max_iter': [25, 50, 100, 200]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [14]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 800, 'max_iter': 200}
0.8632459849074727


# Save the Model

In [15]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'logicreg.sav'
joblib.dump(model, filename)

['logicreg.sav']